In [7]:
from langchain_community.vectorstores import BigQueryVectorSearch
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_vertexai import VertexAI
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
import pandas as pd 
import os

api_key = "your-api-key"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'filename.json'

DATASET = "bq_vectordb"
TABLE = "bq_vectors"
PROJECT_ID = "project-id"

embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True,output_key='answer')

bq_object = BigQueryVectorSearch(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location="US",
    embedding=embedding,
)

In [8]:
llm_openai = ChatOpenAI(model="gpt-4-turbo-2024-04-09",api_key=api_key)
retriever = bq_object.as_retriever()

conversational_retrieval = ConversationalRetrievalChain.from_llm(
    llm=llm_openai,retriever=retriever, memory=memory,verbose=False
)

In [9]:
def QaWithMemory(query):
    return conversational_retrieval.invoke(query)["answer"]

In [15]:
query = """
        What was the rate of growth in AI research publications from 2010 to 2021,
        and which type of AI publication saw the most significant increase in this period?
        """
answer = QaWithMemory(query)

print(answer)

Between 2010 and 2021, the total number of AI research publications more than doubled, growing from 200,000 in 2010 to almost 500,000 in 2021. The category of publications that experienced the most significant increase during this period was repository submissions, which grew 26.6 times.


In [16]:
followup_query = """
             and how might this growth impact the future of AI research priorities?
                """

answer = QaWithMemory(followup_query)

print(answer)

The significant growth in AI research publications from 2010 to 2021, as detailed in the AI Index Report 2023, suggests a few potential impacts on future AI research priorities:

1. **Focus on Emerging Subfields**: Given the sharp increase in publications in areas like pattern recognition and machine learning, future research might prioritize these subfields even more, possibly leading to new breakthroughs and applications. The growth in these areas indicates strong interest and perceived value, likely attracting further investments and talent.

2. **Increased Interdisciplinary Collaboration**: The diverse range of AI fields such as computer vision, data mining, and natural language processing, coupled with growth in cross-industry collaborations, suggests that future AI research might increasingly involve interdisciplinary and cross-sectoral efforts. This could lead to more comprehensive solutions that integrate AI with other fields of study.

3. **Shift in Publication Mediums**: With

In [13]:
query = """What percentage of the world's large language and multimodal models in 2022 were produced by American institutions?"""
answer = QaWithMemory(query)

print(answer)

In 2022, 54% of the large language and multimodal models produced globally originated from American institutions.
